# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for to train an agent to collect bananas!

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
# Watch for changes
%load_ext autoreload
%autoreload 2

from torch.utils.tensorboard import SummaryWriter
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Monkey patch missing attributes for newer numpy versions
if not hasattr(np, "float_"):
    np.float_ = np.float64
    
if not hasattr(np, "int_"):
    np.int_ = np.int64

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
# print current working directory
import os
print(os.getcwd())

/home/oliver/project-showroom/projects/reinforcement-learning/navigation/p1_navigation


In [2]:
env = UnityEnvironment(file_name="Banana_Linux/Banana.x86_64")

Found path: /home/oliver/project-showroom/projects/reinforcement-learning/navigation/p1_navigation/Banana_Linux/Banana.x86_64
Mono path[0] = '/home/oliver/project-showroom/projects/reinforcement-learning/navigation/p1_navigation/Banana_Linux/Banana_Data/Managed'
Mono config path = '/home/oliver/project-showroom/projects/reinforcement-learning/navigation/p1_navigation/Banana_Linux/Banana_Data/MonoBleedingEdge/etc'
Preloaded 'libgrpc_csharp_ext.x64.so'
Unable to preload the following plugins:
	ScreenSelector.so
	libgrpc_csharp_ext.x86.so
	ScreenSelector.so
Logging to /home/oliver/.config/unity3d/Unity Technologies/Unity Environment/Player.log


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment and to initialize **action_size** and **state_size**.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)

state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [8]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: -1.0


### 4. Now train the agent in the banana environment!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

<div style="border-left: 5px solid #2196F3; background-color: #E3F2FD; padding: 10px;">
    <b>ℹ️:</b> The source code for the agent and the neural net model can be found in 'codebase/v2' directory.
</div>


In [ ]:
from collections import deque
import torch
from codebase.v2.dqn_agent import Agent
from tqdm.notebook import tqdm
from datetime import datetime

# Generate a timestamped directory name
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_dir = f"runs/banana_experiment_{timestamp}"

# Initialize TensorBoard writer with a unique directory
writer = SummaryWriter(log_dir=log_dir)

agent = Agent(state_size=state_size, action_size=action_size, seed=20)

def dqn(env, n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.99):
    """Deep Q-Learning with tqdm progress bar updates.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=False)[brain_name]  # reset the environment
        state = env_info.vector_observations[0]
        score = 0

        # Use tqdm to create a progress bar for the steps within an episode.
        with tqdm(total=max_t, desc=f"Episode {i_episode}", leave=False) as pbar:
            for _ in range(max_t):
                # Select an action and let the agent record its Q-values.
                action = agent.act(state, eps)
                
                # Access the Q-values after act() is called.                                
                writer.add_histogram("Q-values", agent.last_q_values, global_step=i_episode)
            
                q_values = agent.last_q_values
                
                env_info = env.step(action)[brain_name]  # send the action to the environment
                next_state = env_info.vector_observations[0]  # get the next state
                reward = env_info.rewards[0]                  # get the reward
                done = env_info.local_done[0]                 # check if episode is finished
                score += reward

                agent.step(state, action, reward, next_state, done)

                # Retrieve the loss from the last learning update.
                loss = agent.last_loss if hasattr(agent, "last_loss") else "N/A"

                if loss is not None: # this is a hack to avoid the error when loss is None
                    writer.add_scalar("Loss", agent.last_loss, global_step=i_episode)
                
                state = next_state

                # Update the progress bar's postfix with the current loss and Q-values.
                pbar.set_postfix(loss=loss, q_values=q_values)
                pbar.update(1)

                writer.add_scalar("Episode/Beta", agent.last_beta, i_episode)

                if done:
                    break

        scores_window.append(score)  # save most recent score
        scores.append(score)         # save most recent score
        eps = max(eps_end, eps_decay * eps)  # decrease epsilon

        # Log episode-level statistics.
        writer.add_scalar("Episode/Average_Score", np.mean(scores_window), i_episode)
        writer.add_scalar("Episode/Score", score, i_episode)
        writer.add_scalar("Episode/Epsilon", eps, i_episode)
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        
        # save every 5 episodes
        if i_episode % 5 == 0:
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')

        if np.mean(scores_window) >= 13.0: # I am stupid, I forgot to set this to 13.0 (from 200) before training the agent, rrrrrr
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'
                  .format(i_episode - 100, np.mean(scores_window)))
            
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    
    writer.close()
    
    return scores

Buffer size: 100000, Batch size: 64, Beta start: 0.4, Beta frames: 100000


In [6]:
scores = dqn(
    env, n_episodes=2000, max_t=300, eps_start=1.0, eps_end=0.01, eps_decay=0.99)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 267	Average Score: 9.53

Episode 268:   0%|          | 0/300 [00:00<?, ?it/s]

Episode 268	Average Score: 9.52

Episode 269:   0%|          | 0/300 [00:00<?, ?it/s]

Episode 269	Average Score: 9.55

...

Episode 453	Average Score: 13.20

Episode 454:   0%|          | 0/300 [00:00<?, ?it/s]

Episode 454	Average Score: 13.18

Episode 455:   0%|          | 0/300 [00:00<?, ?it/s]

KeyboardInterrupt: 

I am an idiot. I forgot to adapt the reward check in my training (taken over from my lunarlander code) from 200 to only 13. Thus I had to terminate the training by myself. Let's hope the weights are properly saved.

But here I insert some **metrics** I recorded with **Tensorboard** to document the training progress:

Development of Average Score / Episode:
![Episode / Average Score](screenshots/Episode_Average_Score.png)
---
Development of Epsilon:
![Episode / Epsilon Value](screenshots/Episode_Epsilon_Score.png)
---
Loss of the Q-Model:
![Loss / Q-Model](screenshots/Loss_QModel.png)
---
Development of Q-Values
![Q Values growing](screenshots/QValues_orange.png)
---

**Explanation**: The bluish curves were from a previous not successful training run. It crashed after several hours (the unity environment stopped responding).

## Let's get our trained agent into the wilds!
Ok, now let's check the weights and if our trained agent can collect enough bananas:

In [ ]:
from codebase.v2.dqn_agent import Agent
import torch

# environment is startet some cells above
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score

bananaCollector = Agent(state_size=state_size, action_size=action_size, seed=20)
bananaCollector.qnetwork_local.load_state_dict(torch.load('checkpoint.pth', weights_only=True))

while True:
    action = bananaCollector.act(state)            # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Buffer size: 100000, Batch size: 64, Beta start: 0.4, Beta frames: 100000
Agent's Device: cuda:0
Score: 20.0


Results: **Wow, wow, wow!!** <b>20</b> is a nice result. Training payed off ;-)